# Agents

In [ ]:
import random
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv

load_dotenv();

## Preparation

In [ ]:
known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description

In [12]:
get_weather_tool = {
    "name": "get_weather",
    "description": "Get the current weather for a specific city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to get weather information for."
            }
        },
        "required": ["city"],
    }
}

TODO3 is city

In [13]:
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
tools = types.Tool(function_declarations=[get_weather_tool])
config = types.GenerateContentConfig(tools=[tools])


In [14]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What's the weather like in Berlin?",
    config=config,
)

In [22]:
def check_function_call(response):
    if response.candidates[0].content.parts[0].function_call:
        function_call = response.candidates[0].content.parts[0].function_call
        print(f"Function to call: {function_call.name}")
        print(f"Arguments: {function_call.args}")
        #  In a real app, you would call your function here:
        #  result = schedule_meeting(**function_call.args)
    else:
        print("No function call found in the response.")
        print(response.text)

## Q2. Adding another tool

In [19]:
set_weather_tool = {
    "name": "set_weather",
    "description": "Set or update the temperature for a specific city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to set the temperature for."
            },
            "temp": {
                "type": "number",
                "description": "Temperature in Celsius to set for the city."
            }
        },
        "required": ["city", "temp"],
    }
}

In [20]:
tools = types.Tool(function_declarations=[get_weather_tool, set_weather_tool])
config = types.GenerateContentConfig(tools=[tools])
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Set the temperature in Warsaw to 22.5 degrees.",
    config=config,
)

In [ ]:
check_function_call(response)

Function to call: set_weather
Arguments: {'city': 'Warsaw', 'temp': 22.5}


## Q3. Install FastMCP

🏎️  FastMCP version: 2.10.5 

## Q4. Simple MCP Server

TODO is stdio

## Q5. Protocol

```json
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}
```

## Q6. Client

```python
[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}}, 'required': ['city', 'temp'], 'type': 'object'}, outputSchema=None, annotations=None, meta=None)]
```